In [ ]:
import tensorflow as tf
import numpy as np

## Simple Math

In [ ]:
inputs = np.random.rand(100, 20)
weights = np.random.rand(20, 1)

In [ ]:
graph_1 = tf.Graph()
with graph_1.as_default():
    x = tf.placeholder(dtype=tf.float64, name='x')
    w = tf.placeholder(dtype=tf.float64, name='w')
    y = tf.matmul(x, w)

In [ ]:
# tf.global_variables_initializer()
with tf.Session(graph=graph_1) as sess:
    y_val = sess.run(y, feed_dict={x: inputs, w: weights})

In [ ]:
y_val.shape

## LSTM Trials

In [ ]:
sequence_inputs = list()
for i in range(100):
    sequence_inputs.append(np.random.rand(20, 50))

# sequence_inputs = np.random.rand(100, 20, 50)

In [ ]:
graph_2 = tf.Graph()

with graph_2.as_default():
    num_lstm_units = 128
    inputs_series = tf.placeholder(dtype=tf.float64, name='inputs_series', shape=(100, 20, 50))
    init_state = tf.zeros([100], tf.float64, name='init_state') 
#     init_state = tf.placeholder(dtype=tf.float64, name='init_state')
    lstm_1 = tf.contrib.rnn.BasicLSTMCell(num_lstm_units)
    
    states_series, current_state = \
        tf.nn.dynamic_rnn(lstm_1, inputs_series, init_state, dtype=tf.float64)

In [ ]:
with tf.Session(graph=graph_2) as sess:
    sess.run(tf.global_variables_initializer())
    states_series, current_state = sess.run([states_series, current_state], 
                                            feed_dict={inputs_series: sequence_inputs})

In [ ]:
current_state.c.shape, current_state.h.shape

In [ ]:
states_series.shape